# Scratch - Testing Google Drive

In [ ]:
import pycamhd as camhd
import numpy as np

In [ ]:
fileid = 'gdid://1RWPD7ME8WsrFjr-peGAz8KQekueA9Nz-'
fileurl = 'https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2017/08/29/CAMHDA301-20170829T001500.mov'

In [ ]:
%%time
moov_atom = camhd.get_moov_atom(fileid)
frame_count = camhd.get_frame_count(fileid, moov_atom)

In [ ]:
%%time
moov_atom = camhd.get_moov_atom(fileurl)
frame_count = camhd.get_frame_count(fileurl, moov_atom)

In [ ]:
frame = camhd.get_frame(fileid, 1000)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rc('figure', figsize=(12, 12))
imgplot = plt.imshow(frame)

In [ ]:
from dask import delayed
import dask.array as dsa

In [ ]:
frame_numbers = list(np.arange(1,8000))

In [ ]:
delayed_frame_list = []
delayed_moov_atom = delayed(camhd.get_moov_atom)(fileurl)         
for frame_number in frame_numbers:
    delayed_frame = delayed(camhd.get_frame)(fileurl, frame_number, 'gray16le', delayed_moov_atom)
    delayed_frame_list.append(dsa.from_delayed(delayed_frame, (1080, 1920), np.uint16))
delayed_url_array = dsa.stack(delayed_frame_list)
delayed_url_array

In [ ]:
%%time
test = delayed_url_array.max().compute()

In [ ]:
delayed_frame_list = []
delayed_moov_atom = delayed(camhd.get_moov_atom)(fileid)         
for frame_number in frame_numbers:
    delayed_frame = delayed(camhd.get_frame)(fileid, frame_number, 'gray16le', delayed_moov_atom)
    delayed_frame_list.append(dsa.from_delayed(delayed_frame, (1080, 1920), np.uint16))
delayed_id_array = dsa.stack(delayed_frame_list)
delayed_id_array

In [ ]:
%%time
test = delayed_id_array.max().compute()

#### Start a Dask cluster

In [ ]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=2)
cluster

In [ ]:
from dask.distributed import Client
client = Client(cluster)
client